In [1]:
from DataFetcher import DataFetcher
from USTs import USTs
from Curves import ParCurves, calibrate_mles
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime
import plotly.graph_objects as go

In [ ]:
usts = USTs(date=datetime.date(2025, 6, 26))

ust_set = usts.get_current_UST_set(settlement_date=datetime.datetime(2025, 6, 27),
                                   get_ytms=True,
                                   include_FRNs=False,
                                   include_TIPS=False,
                                   include_outstanding=False)

In [4]:
ust_set

,Security type,Rate,Maturity date,End of day,Security term,Issue date,Years to maturity,UST label,EOD YTM,Duration,Rank
Cusip,,,,,,,,,,,
912828XZ8,Note,2.750,2025-06-30,99.968750,7-Year,2018-07-02,0.010959,2.750% Jun-25,6.559941,0.007980,84
91282CHL8,Note,4.625,2025-06-30,99.968750,2-Year,2023-06-30,0.010959,4.625% Jun-25,8.400867,0.007910,24
912828ZW3,Note,0.250,2025-06-30,99.968750,5-Year,2020-06-30,0.010959,0.250% Jun-25,4.078317,0.008077,58
912797PU5,Bill,0.000,2025-07-01,99.954778,17-Week,2025-03-04,0.013699,0.000% Jul-25,4.128374,0.016467,17
912797NX1,Bill,0.000,2025-07-03,99.932333,26-Week,2025-01-02,0.019178,0.000% Jul-25,4.119197,0.016240,19
...,...,...,...,...,...,...,...,...,...,...,...
912810UA4,Bond,4.625,2054-05-15,96.687500,30-Year,2024-05-15,28.904110,4.625% May-54,4.840632,15.526043,5
912810UC0,Bond,4.250,2054-08-15,90.906250,30-Year,2024-08-15,29.156164,4.250% Aug-54,4.833356,15.708498,4
912810UE6,Bond,4.500,2054-11-15,94.875000,30-Year,2024-11-15,29.408219,4.500% Nov-54,4.829236,15.752849,3


In [4]:
usts.plot_ust_curve(bspline_curve=True, univariate_spline=True, mles_spline=True)

In [5]:
fig = go.Figure()
data = ust_set[(ust_set['Duration'] > 9) & (ust_set['Duration'] < 14)]
tenors = ['20-Year', '30-Year']

for tenor in tenors:
    plot_data = data[data['Security term'] == tenor]
    fig.add_trace(go.Scatter(
        x=plot_data['Duration'],
        y=plot_data['EOD YTM']/100,
        mode='markers+text',
        marker=dict(size=10),
        customdata=plot_data.reset_index().values,
        name=tenor,
        text=plot_data['Maturity date'].dt.strftime('%b-%y'),
        textposition='top center',
        textfont=dict(size=10)
    ))

fig.update_layout(
    title='Selected Treasury YTMs and Durations',
    width=1000,
    height=600,
    margin=dict(l=20, r=20, t=50, b=20),
    xaxis_title='Duration (Years)',
    yaxis_title='End-of-Day Yield to Maturity (YTM)',
    yaxis_tickformat='.2%',
    legend_title_text='Security Tenors',
    template='plotly_dark'
)

In [6]:
full_df = usts.get_initial_screening_set()

In [7]:
full_df

,Duration exposure,Current spread,Par curve spread,Long bond,Tenor,YTM,Short bond,Tenor,YTM
0,Long duration,10.4 bps,0.5 bps,2.875% May-43,30-Year,4.833758,1.125% May-40,20-Year,4.729779
1,Long duration,10.5 bps,1.7 bps,3.125% Aug-44,30-Year,4.862613,1.375% Nov-40,20-Year,4.757422
2,Long duration,10.5 bps,2.0 bps,4.375% Nov-39,30-Year,4.601410,3.500% Feb-39,30-Year,4.496092
3,Long duration,16.9 bps,9.0 bps,4.750% Feb-41,30-Year,4.665072,3.500% Feb-39,30-Year,4.496092
4,Long duration,10.8 bps,4.5 bps,3.125% Aug-44,30-Year,4.862613,1.875% Feb-41,20-Year,4.754230


In [8]:
yields, prices = usts.get_cusip_timeseries(CUSIPs=['912810RB6', '912810SR0'],
                          start_date=datetime.date(2025, 6, 10),
                          end_date=datetime.date(2025, 6, 26))
yields

,912810SR0,912810RB6,Spread
Date,,,
2025-06-10,4.885072,4.980880,0.095808
2025-06-11,4.834285,4.946301,0.112016
2025-06-12,4.756544,4.874378,0.117834
2025-06-13,4.836695,4.950326,0.113631
2025-06-16,4.888736,5.001277,0.112541
2025-06-17,4.810358,4.922402,0.112044
2025-06-18,4.819417,4.922833,0.103416
2025-06-20,4.817287,4.923479,0.106192
2025-06-23,4.782582,4.892318,0.109736


In [9]:
price_data = DataFetcher().fetch_historical_UST_data(date=datetime.datetime(2025, 7, 7))
price_data

,Cusip,Security type,Rate,Maturity date,Buy,Sell,End of day
0,912797PZ4,Bill,0.000,2025-07-08,0.000000,99.988389,100.000000
1,912797LW5,Bill,0.000,2025-07-10,0.000000,99.965000,99.976722
2,912797QA8,Bill,0.000,2025-07-15,99.907000,99.906667,99.918528
3,912797PE1,Bill,0.000,2025-07-17,99.884167,99.883889,99.895500
4,912797QB6,Bill,0.000,2025-07-22,99.826458,99.825833,99.838222
...,...,...,...,...,...,...,...
452,91282CKM2,FRN,4.492,2026-04-30,100.067969,100.039809,100.049863
453,91282CLA7,FRN,4.525,2026-07-31,100.088736,100.057281,100.067762
454,91282CLT6,FRN,4.548,2026-10-31,100.119985,100.100637,100.107085
455,91282CMJ7,FRN,4.440,2027-01-31,99.960241,99.941156,99.936577
